# Sample for a Fisher Market with UMP with homogeneous affine constraints

- the current implementation is for concept proof only,
  it is far from efficient and scalable to large-scale problems

In [1]:
using Pkg
Pkg.activate("../")

  Activating project at `~/workspace/ExchangeMarket.jl/scripts`


In [2]:
using Revise
using SparseArrays, LinearAlgebra
using JuMP, MosekTools
import MathOptInterface as MOI
using Plots, LaTeXStrings, Printf, Random, GraphRecipes
using CSV, DataFrames, Graphs, Distributions

using ExchangeMarket

include("../tools.jl")
include("../plots.jl")
include("setup.jl")
switch_to_pdf(;)


:pdf

## Random instance

In [3]:
function generate_random_graph(nᵥ)
    # or create a small random graph
    E = []
    for i in 1:nᵥ
        for j in 1:nᵥ
            if i != j
                push!(E, (i, j))
            end
        end
    end
    nₑ = length(E)
    adj = sparse(first.(E), last.(E), ones(length(E)), nᵥ, nᵥ)
    # assign capacities and random revenues
    # the first element is for the output flow
    c = rand(Float64, nₑ + 1)
    r = rand(Uniform(1.0, 10.0), nₑ + 1)
    return adj, E, c, r, nᵥ, nₑ
end

adj, E, c, r, nᵥ, nₑ = generate_random_graph(4)


(sparse([2, 3, 4, 1, 3, 4, 1, 2, 4, 1, 2, 3], [1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 4, 4), Any[(1, 2), (1, 3), (1, 4), (2, 1), (2, 3), (2, 4), (3, 1), (3, 2), (3, 4), (4, 1), (4, 2), (4, 3)], [0.11098350530714718, 0.6312691049886633, 0.26277161031283547, 0.32462217548329697, 0.05486579908642786, 0.5422263484025841, 0.06349528364222179, 0.08248477470886806, 0.19345897472271611, 0.7181125526196892, 0.5791906015258148, 0.5620087312007507, 0.19272219626589837], [3.2535776025043184, 5.623716451005054, 9.35282507086276, 6.995462663837024, 2.363574128874775, 1.1756502314713158, 4.21802189053963, 5.180036365306834, 3.4074081121537687, 4.65656309912786, 3.202162020603367, 2.855187289658669, 4.612202738398566], 4, 12)

In [37]:
m = 10
n = length(E)
stpairs = [
    zip(fill(1, m), rand(1:nᵥ, m))...
]
vec_const_x = []
for (s, t) in stpairs
    md, A, _... = ExchangeMarket.__query_Abc_standard(s, t, adj, E, nᵥ)
    mₐ, nₐ = size(A)
    push!(vec_const_x, LinearConstr(mₐ, nₐ, A, zeros(mₐ)))
end
ρ = 0.5
constr_p = LinearConstr(1, n, ones(1, n), [1.0])

f0 = FisherMarket(
    m, n; ρ=ρ,
    constr_x=vec_const_x, constr_p=constr_p,
    bool_unit=true, bool_unit_wealth=true,
    scale=10.0, sparsity=0.95
)

FisherMarket initialization started...
FisherMarket cost matrix initialized in 0.00011396408081054688 seconds
FisherMarket initialized in 0.000164031982421875 seconds


FisherMarket{Float64}(10, 12, [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.006374582858360694, 0.047222993494899265, 0.1325479751695242, 0.1612630654900847, 0.16467359910596133, 0.07578967849136997, 0.13070238311665255, 0.005282973243341747, 0.15828207487109827, 0.11786067415870717], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], ExchangeMarket.var"#20#31"(Core.Box([0.7580404886960546 23.888319995115484 … 6.476824583760228 0.8131698731973092; 1.5013522433103244 2.779064568908126 … 10.0 7.279222226351337; … ; 4.849539347158185 2.0847480287404316 … 4.235276066272745 3.9185635106781946; 0.5590087824933776 16.69182768597185 … 0.0 7.970326197841494])), ExchangeMarket.var"#24#35"{Float64}(0.5, Core.Box([0.7580404886960546 23.888319995115484 … 6.476824583760228 0.8131698731973092; 1.50135224331032

In [38]:
# define optimization arguments
method = HessianBar
kwargs = Dict(
    :tol => 1e-10,
    :maxiter => 20,
    :optimizer => AFCESConic,
    # :option_mu => :pred_corr,
    :option_mu => :nothing,
    :option_step => :affinesc,
    :linsys => :direct_affine, # only support direct for now
)

Dict{Symbol, Any} with 6 entries:
  :maxiter     => 20
  :tol         => 1.0e-10
  :optimizer   => ResponseOptimizer(__af_conic_log_response_ces, :linconicaffin…
  :option_mu   => :nothing
  :linsys      => :direct_affine
  :option_step => :affinesc

In [ ]:
f1 = copy(f0)
p₀ = ones(n) * sum(f1.w) ./ (n)
if typeof(f1.c) <: SparseMatrixCSC
    x₀ = similar(f1.c)
    x₀.nzval .= 1.0
else
    x₀ = ones(n, m) ./ m
end
f1.x .= x₀
f1.p .= p₀
alg = method(
    n, m, p₀;
    linconstr=constr_p,
    μ=1e-6,
    kwargs...
)
traj = opt!(
    alg, f1;
    keep_traj=true,
    maxiter=50,
    maxtime=500.0,
    loginterval=1,
    tol=1e-12
)
validate(f1, alg)

FisherMarket initialization started...
FisherMarket cost matrix initialized in 7.796287536621094e-5 seconds
FisherMarket initialized in 0.00011992454528808594 seconds


HessianBar{Float64}(12, 10, [0.08333333333333331, 0.08333333333333331, 0.08333333333333331, 0.08333333333333331, 0.08333333333333331, 0.08333333333333331, 0.08333333333333331, 0.08333333333333331, 0.08333333333333331, 0.08333333333333331, 0.08333333333333331, 0.08333333333333331], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.08333333333333331, 0.08333333333333331, 0.08333333333333331, 0.08333333333333331, 0.08333333333333331, 0.08333333333333331, 0.08333333333333331, 0.08333333333333331, 0.08333333333333331, 0.08333333333333331, 0.08333333333333331, 0.08333333333333331], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0], [0.0], [0.604149828911257, 0.5336085663333506, 0.9628985358994283, 0.5460329380374557, 0.8098860312750314, 0.9578878612892814, 0.9914835641805719, 0.19217068642025237, 0.8603940121903297, 0.7538272613882555, 0.7463191704063897, 0.05979467827914209], 1.0e-6, -1.0e6, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0,

## Test flow conservation

In [42]:
for i in 1:m
    _f = f1.x[:, i]
    _A = f1.constr_x[i].A
    @info "flow conservation residual:" _A * _f .|> abs |> sum
end

┌ Info: flow conservation residual:
│   (_A * _f .|> abs) |> sum = 2.400012697259535e-12
└ @ Main /Users/brent/Documents/recherche/25-ipm-auction-fisher/ExchangeMarket.jl/scripts/fisher/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y150sZmlsZQ==.jl:4
┌ Info: flow conservation residual:
│   (_A * _f .|> abs) |> sum = 1.580271265406996e-11
└ @ Main /Users/brent/Documents/recherche/25-ipm-auction-fisher/ExchangeMarket.jl/scripts/fisher/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y150sZmlsZQ==.jl:4
┌ Info: flow conservation residual:
│   (_A * _f .|> abs) |> sum = 5.701275411531246e-9
└ @ Main /Users/brent/Documents/recherche/25-ipm-auction-fisher/ExchangeMarket.jl/scripts/fisher/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y150sZmlsZQ==.jl:4
┌ Info: flow conservation residual:
│   (_A * _f .|> abs) |> sum = 4.786434322681998e-7
└ @ Main /Users/brent/Documents/recherche/25-ipm-auction-fisher/ExchangeMarket.jl/scripts/fisher/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y